In [ ]:
from morpholog import Morpholog 
import re
from collections import Counter, defaultdict
import pickle
import json

with open(r"train_data\train.txt", "r", encoding="utf-8") as file:
    text = file.read()

In [ ]:
class Morphology_Tok:
    def __init__(self, text):
        self.text = text

    def tokenize_with_morpholog(self):
            analyzer = Morpholog()
            roots = []
            endings = []

            for word in self.text:
                root = analyzer.get_roots(word)
                ending = analyzer.get_ending(word)

                if root:
                    cleaned_root = root[0].strip("',\"().")
                    roots.append(cleaned_root)

                if ending:
                    cleaned_ending = ending.strip("',\"().+")
                    endings.append(cleaned_ending)

            return roots, endings
    
# morph = Morphology_Tok(text)
# roots, endings = morph.tokenize_with_morpholog()
# for root in roots:
#      print(root)


In [ ]:
class PairFinder:
    def __init__(self, threshold):
        self.threshold = threshold
        self.pair_counts = {}
    
    def find_and_return_pairs(self, text):
        # Инициализирует словарь для хранения парных частот
        for i in range(len(text) - 1):
            pair = text[i:i+2]
            if all(c.isalpha() for c in pair):  # Рассматриваем только алфавитные пары
                self.pair_counts[pair] = self.pair_counts.get(pair, 0) + 1
        
        # Создайте набор часто встречающихся пар
        frequent_pairs = {pair for pair, count in self.pair_counts.items() if count >= self.threshold}
        
        # Возвращайте только часто встречающиеся пары
        return frequent_pairs

# # Пример использования
# text = "Привет, это пример текста с некоторыми парами, такими как 'ab', 'cd' и 'ef'."
# pair_finder = PairFinder(threshold=3)
# result = pair_finder.find_and_return_pairs(text)
# print("Часто встречающиеся пары:", result)


In [ ]:
class Character_Tok:
    def __init__(self, text):
        self.unique_characters = set()
        self.text = text

    def collect_unique_characters(self):
        for word in self.text:
            self.unique_characters.update(set(word))

    def get_unique_characters(self):
        return self.unique_characters
    
# collector = Character_Tok(text)
# collector.collect_unique_characters()
# unique_characters = collector.get_unique_characters()
# print(unique_characters)

In [ ]:
class CustomTokenizer:
    def __init__(self, text):
        self.text = text
        self.morphology_tok = Morphology_Tok(text)
        self.character_tok = Character_Tok(text)
        self.pair_finder = PairFinder(threshold=3)
        self.symbols = ['А', 'Б', 'В', 'Г', 'Д', 'Е', 'Ё', 'Ж', 'З', 'И', 'Й', 'К', 'Л', 'М', 'Н', 'О', 'П', 'Р',
                        'С', 'Т', 'У', 'Ф', 'Х', 'Ц', 'Ч', 'Ш', 'Щ', 'Ъ', 'Ы', 'Ь', 'Э', 'Ю', 'Я',
                        'а', 'б', 'в', 'г', 'д', 'е', 'ё', 'ж', 'з', 'и', 'й', 'к', 'л', 'м', 'н', 'о', 'п', 'р', 'с', 'т', 'у', 'ф', 'х', 'ц', 'ч', 'ш', 'щ', 'ъ', 'ы', 'ь', 'э', 'ю', 'я',
                        '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '!', '"', '#', '$', '%', '&', "'", '(', ')', '*', '+', ',']
        
        # Объединяем все токены в один словарь
        roots, endings = self.morphology_tok.tokenize_with_morpholog()
        frequent_pairs = self.pair_finder.find_and_return_pairs(self.text)
        unique_characters = self.character_tok.get_unique_characters()
        self.all_tokens = roots + endings + list(frequent_pairs) + list(unique_characters) + self.symbols


        print(f"Корни: {roots}")
        print(f"Окончания: {endings}")
        print(f"Пары: {frequent_pairs}")
        print(f"Символы: {unique_characters}")
        print(f"Символы: {self.symbols}")

        # Присваиваем значения токенам
        self.token_values = {token: i for i, token in enumerate(self.all_tokens)}

    def encode_token(self, token):
        return [self.token_values[token]]  # Получаем числовое значение токена
    
    def decode_token(self, encoded_value):
        return self.all_tokens[encoded_value[0]]   # Преобразуем числовое значение обратно в строку
    
    def tokenize(self):
        encoded_tokens = [self.encode_token(token) for token in self.all_tokens]
        return encoded_tokens
    
    def save_to_pickle(self, filename):
        with open(filename, 'wb') as f:
            pickle.dump(self.token_values, f)
        print(f"Токенизатор сохранен в файл {filename} в формате pkl.")
    
    def save_to_json(self, filename):
        with open(filename, 'w',  encoding='utf-8') as f:
            json.dump(self.token_values, f, ensure_ascii=False, indent=4)
        print(f"Токенизатор сохранен в файл {filename} в формате json.")


with open(r"train_data\train.txt", "r", encoding="utf-8") as file:
    text = file.read()

#Обучение токенизатора
tokenizer = CustomTokenizer(text)
tokenizer.save_to_pickle(r"gpt\tokenizer.pkl")
tokenizer.save_to_json(r"gpt\tokenizer.json")


In [ ]:
# class CombinedTokenizer:
#     def __init__(self, text, num_merges):
#         self.text = text
#         self.num_merges = num_merges
#         self.combined_vocab = None  # Инициализация атрибута

#     def tokenize(self):
#         # Инициализация и обработка текста морфологическим токенизатором
#         morphology_tok = Morphology_Tok(self.text)
#         roots, endings = morphology_tok.tokenize_with_morpholog()

#         character_tok = Character_Tok(self.text)
#         character_tok.collect_unique_characters()
#         chars = character_tok.get_unique_characters()

#         print(f"Корни: {roots}")
#         print(f"Окончания: {endings}")
#         print(f"BPE словарь: {list(bpe_tok.vocab.keys())}")
#         print(f"Символы: {list(chars)}")

#         # Объединение результатов в один словарь
#         self.combined_vocab = set(roots + endings + list(bpe_tok.vocab.keys()) + list(chars))

#         self.token_to_id = {token: id for id, token in enumerate(self.combined_vocab)}
#         self.id_to_token = {id: token for token, id in self.token_to_id.items()}

#         # Преобразование токенов в числовые идентификаторы
#         token_ids = [self.token_to_id[token] for token in self.combined_vocab]
#         return token_ids
    
#     def save_vocab(self):
#         # Сохранение словаря в файл
#         with open(r'gpt/token_to_id.pkl', 'wb') as file:
#             pickle.dump(self.token_to_id, file)
#         print("Словарь успешно сохранен.")
        
#         # Сохранение обратного словаря в файл
#         with open(r'gpt/id_to_token.pkl', 'wb') as file:
#             pickle.dump(self.id_to_token, file)
#         print("Обратный словарь успешно сохранен.")

#     def save_vocab_json(self):
#         # Сохранение словаря token_to_id в файл JSON
#         with open(r'gpt/token_to_id.json', 'w', encoding='utf-8') as file:
#             json.dump(self.token_to_id, file, ensure_ascii=False, indent=4)
#         print("Словарь token_to_id успешно сохранен в формате JSON.")

#         # Сохранение обратного словаря id_to_token в файл JSON
#         with open(r'gpt/id_to_token.json', 'w', encoding='utf-8') as file:
#             json.dump(self.id_to_token, file, ensure_ascii=False, indent=4)
#         print("Обратный словарь id_to_token успешно сохранен в формате JSON.")
    
# def test_tokenizer(tokenizer):
#     tokens = tokenizer.tokenize()
#     print(f"Токены: {tokens}")
#     print(f"Количество токенов: {len(tokens)}")

# # Пример использования
# text = "Пример текста для токенизации"
# num_merges = 100  # Количество итераций для BPE
# tokenizer = CombinedTokenizer(text, num_merges)
# token_ids = tokenizer.tokenize()

# # Сохранение словарей
# tokenizer.save_vocab()
# tokenizer.save_vocab_json()

# # Сохранение экземпляра токенизатора в файл
# with open(r'gpt/tokenizer.pkl', 'wb') as file:
#     pickle.dump(tokenizer, file)
# print("Токенизатор успешно сохранен.")

In [90]:
# Загрузка токенизатора из файла формата pkl
with open(r'gpt\tokenizer.pkl', 'rb') as file:
    tokenizer_pkl = pickle.load(file)

# Загрузка токенизатора из файла формата json
with open(r'gpt\tokenizer.json', 'r', encoding='utf-8') as file:
    data = json.load(file)
    tokenizer_json = data['tokenizer']

# Создание экземпляра класса CustomTokenizer для JSON
class CustomTokenizer:
    def tokenize(self, text):
        # Реализация метода токенизации
        return text.split()  # Пример простой токенизации разделением по пробелам

tokenizer = CustomTokenizer()

# Пример тестирования токенизатора
text_to_tokenize = "Пример текста для токенизации."

tokens_pkl = tokenizer_pkl.tokenize(text_to_tokenize)
tokens_json = tokenizer_json.tokenize(text_to_tokenize)

print("Токены (pkl):", tokens_pkl)
print("Токены (json):", tokens_json)

KeyError: 'tokenizer'